In [ ]:
%matplotlib inline
%load_ext autoreload

In [ ]:
from tifresi import pyplot as plt
import numpy as np

In [ ]:
# Select a test signal
import librosa
filename = librosa.util.example('brahms')

# Load the signal

In [ ]:
from tifresi.utils import load_signal
y, sr = load_signal(filename)
# Apply some light preprocessing
from tifresi.utils import preprocess_signal
cut_len = 512*128
y = preprocess_signal(y)[10*cut_len:11*cut_len]
# Plot the signal in the time domain
n = 512*128

t = np.arange(n)/sr*1000
plt.figure(figsize=(4, 2.5))
plt.plot(t, y[:n])
plt.xlim([0, n/sr*1000])
plt.xlabel('Time [ms]');

In [ ]:
from IPython.display import display, Audio
display(Audio(y, rate=sr))

# Define the main parameters
* The number of frequency channel for the stft `stft_channels`
* The hop_size in time `hop_size`

In [ ]:
# Here we use the default paramters. We recommend using them.
from tifresi.hparams import HParams
stft_channels = HParams.stft_channels # 1024
stft_channels = 2048
# hop_size = HParams.hop_size # 256
hop_size = 32
hop_size = 256


# Build the time frequency system
The system will be used for the following operation:
* Compute the STFT
* Compute the spectrogram and the phase gradients
* Reconstruct the phase from the phase gradients
* Invert the STFT

In [ ]:
from tifresi.stft import PhaseGradTF
stft_system = PhaseGradTF(hop_size=hop_size, stft_channels=stft_channels)

In [ ]:
magSpectrogram, tgrad, fgrad = stft_system.mag_phase_grad(y)
np.max(magSpectrogram)

In [ ]:
dynamic_range_dB = 50

# We use the log spectrogram as we have a logarithmic perception of sound energy.
from tifresi.transforms import log_spectrogram, inv_log_spectrogram
logSpectrogram= log_spectrogram(magSpectrogram, dynamic_range_dB=dynamic_range_dB)
plt.figure(figsize=(10, 7))
plt.subplot(3, 1, 1)
plt.imshow(logSpectrogram + dynamic_range_dB, cmap="afmhot_r")
plt.title('Log spectrogram')
plt.colorbar()
plt.subplot(3, 1, 2)
clim = np.max(np.abs(tgrad))
plt.imshow(tgrad, cmap="seismic", clim=(-clim, clim))
plt.colorbar()
plt.title('Time gradient')
plt.subplot(3, 1, 3)
clim = np.max(np.abs(fgrad))
plt.imshow(fgrad, cmap="seismic", clim=(-clim, clim))
plt.title('Frequency gradient')
plt.colorbar()
plt.tight_layout()


# Invert the representation

In [ ]:
magSpectrogramR = inv_log_spectrogram(logSpectrogram)

In [ ]:
y_reconstructed = stft_system.invert_mag_phase_grad(magSpectrogramR, tgrad, fgrad)


In [ ]:
display(Audio(y_reconstructed, rate=sr))